In [1]:
import pandas as pd
from astroquery.gaia import Gaia 

# Calculo de grilla de frecuencias

- Calcular minimo y maximos de frecuencias/periodos de cada tipo, con el periodo de catálogo de Gaia, **segun el dataset**.

In [2]:
data_global = pd.read_pickle("../../PeriodogramsGaia/dataset/data_global.pkl")

In [3]:
def min_max_freq(data_global):
    grid = pd.DataFrame(columns=['low-frecuency', 'high-frequency'], index = data_global.type.unique())
    for type_lc in data_global.groupby('type'):
        name_type = list(type_lc[1].type)[0]
        high = 1/type_lc[1].pf.min()
        low = 1/type_lc[1].pf.max()
        grid.loc[name_type] = [low, high]

    return grid

In [4]:
grid = min_max_freq(data_global)

In [5]:
grid

,low-frecuency,high-frequency
ECL,0.01295,2.665704
RR,1.243939,2.887145


### Mejor opción

- Calcular minimo y maximos de frecuencias/periodos de cada tipo, con el periodo de catálogo de Gaia, segun **tablas de Gaia** con una consulta.

In [6]:

def min_max_freq_tables(tables):
    grid = pd.DataFrame(columns=['low-frecuency', 'high-frequency'], index = tables.keys())
    for type, table in zip(tables.keys(), tables.values()):
        if type == "ECL":
            query = f"""
                SELECT TOP 1 source_id, frequency
                FROM gaiadr3.{table}
                WHERE frequency IS NOT NULL
                ORDER BY frequency ASC

                UNION

                SELECT TOP 1 source_id, frequency
                FROM gaiadr3.{table}
                WHERE frequency IS NOT NULL
                ORDER BY frequency DESC

                """
            
        else:
            query = f"""
                SELECT TOP 1 source_id, pf
                FROM gaiadr3.{table}
                WHERE pf IS NOT NULL
                ORDER BY pf ASC

                UNION

                SELECT TOP 1 source_id, pf
                FROM gaiadr3.{table}
                WHERE pf IS NOT NULL
                ORDER BY pf DESC
                """
            
        
        job = Gaia.launch_job(query)
        info = job.get_results().to_pandas("source_id")
        
        if type == "RR":
            info['frequency'] = 1/info.pf
            info.drop('pf', axis=1, inplace=True)
            high = info.frequency.iloc[1]
            low = info.frequency.iloc[0]
        else:
            high = info.frequency.iloc[0]
            low = info.frequency.iloc[1]
        grid.loc[type] = [low, high]

    return grid

    


In [7]:
tables = {'ECL':"vari_eclipsing_binary",'RR':"vari_rrlyrae"}
grid = min_max_freq_tables(tables)
grid

,low-frecuency,high-frequency
ECL,0.001473,4.999929
RR,1.00061,4.973236
